In [1]:
import pickle
import numpy as np
import pandas as pd

In [34]:
# with open("raw_sensor_data.pkl", "rb") as f:
#     sensor_data = pickle.load(f)

with open("imputed_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [3]:
def build_comparison_df(sensor_data, mat, sensor):
    data_lengths = []
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_data_len = sensor_data[f"mat_{
            mat}"][sensor][heater_idx].shape[0]
        data_lengths.append(heater_data_len)

    min_len = min(data_lengths)
    heater_dict = {}
    for heater_idx in sensor_data[f"mat_{mat}"][sensor]:
        heater_dates = sensor_data[f"mat_{
            mat}"][sensor][heater_idx]["Date"].to_numpy()
        heater_data_len = len(heater_dates)
        if heater_data_len > min_len:
            heater_dates = heater_dates[:min_len]
        heater_dict[heater_idx] = heater_dates

    for heater_idx in range(1, 10):
        diff_seconds = (heater_dict[0] - heater_dict[heater_idx]
                        ).astype("timedelta64[s]").astype(np.int32)
        heater_dict[f"d 0-{heater_idx}"] = diff_seconds

    df = pd.DataFrame(heater_dict)
    return df

In [4]:
def impute_data_mean(df, after):
    df_copy = df.copy()
    line = df_copy.iloc[after:after+2].mean()

    line["Sensor Index"] = int(line["Sensor Index"])
    line["Sensor ID"] = int(line["Sensor ID"])
    line["Time Since PowerOn"] = int(line["Time Since PowerOn"])
    line["Real time clock"] = int(line["Real time clock"])
    line["Heater Profile Step Index"] = int(line["Heater Profile Step Index"])
    line["Scanning Mode Enabled"] = int(line["Scanning Mode Enabled"])
    line["Scanning Cycle Index"] = int(line["Scanning Cycle Index"])
    line["Label Tag"] = int(line["Label Tag"])
    line["Error Code"] = int(line["Error Code"])
    line["Date"] = line["Date"].floor("S")

    index = df_copy.iloc[after:after+2].index
    index = int((index[0] + index[1]) / 2)
    df_copy.loc[index] = line
    df_copy.sort_index(inplace=True)
    return df_copy

In [5]:
MATRIX = 0

In [35]:
for i in range(7, 8):
    df = build_comparison_df(sensor_data, MATRIX, i)
    df.to_excel(f"mat_{MATRIX}_s{i}_diff.xlsx")
    break

In [30]:
sensor_data[f"mat_{MATRIX}"][0][8].iloc[6972:6976]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
303337,0,355892538,75581257,1729766537,37.944469,933.719727,18.580612,8354921.0,8,1,1,0,0,2024-10-24 13:42:17
303379,0,355892538,75592054,1729766548,38.034630,933.747925,18.523739,8329435.5,8,1,1,0,0,2024-10-24 13:42:28
303455,0,355892538,75613874,1729766570,37.899391,933.752136,18.551878,8432321.0,8,1,1,0,0,2024-10-24 13:42:50
303500,0,355892538,75624639,1729766581,37.936958,933.721619,18.554167,8393443.0,8,1,1,0,0,2024-10-24 13:43:01


In [32]:
sensor = 7
heater = 6
after = 3461
df_copy = impute_data_mean(sensor_data[f"mat_{MATRIX}"][sensor][heater], after)
sensor_data[f"mat_{MATRIX}"][sensor][heater] = df_copy.copy()
df_copy.iloc[after-1:after+4]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
262186,7,355868473,65315780,1729756271,37.268105,935.165161,23.132088,537815.1250,6,1,1,0,0,2024-10-24 10:51:11
262259,7,355868473,65334691,1729756289,36.984169,935.135132,23.507763,539515.2500,6,1,1,0,0,2024-10-24 10:51:29
262332,7,355868473,65353598,1729756308,36.878313,935.135986,23.787433,540585.4375,6,1,1,0,0,2024-10-24 10:51:48
262406,7,355868473,65372505,1729756327,36.772457,935.136841,24.067102,541655.6250,6,1,1,0,0,2024-10-24 10:52:07
262487,7,355868473,65391247,1729756346,36.677811,935.183716,24.151482,548179.8750,6,1,1,0,0,2024-10-24 10:52:26


In [33]:
with open("imputed_sensor_data.pkl", "wb") as f:
    pickle.dump(sensor_data, f)